In [1]:
from py_ecc.bn128 import G1, G2, multiply, add, neg, curve_order, Z1, pairing
import galois
from functools import reduce
import numpy as np
import random

In [2]:
def generate_powers_of_tau(tau, degree, point):
    return [multiply(point, int(tau ** i)) for i in range(degree + 1)]

def inner_product(ec_points, coeffs):    
    # Check if number of ec_points equal to the one of coeffs
    assert len(coeffs) == len(ec_points), "Check failed"

    return reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(ec_points, coeffs)), Z1)

In [3]:
print("Initializing a large field, this may take a while...")
p = curve_order
GF = galois.GF(p)
print("Initialization completed. Start computation ...")

Initializing a large field, this may take a while...
Initialization completed. Start computation ...


In [4]:
# Trusted setup 
print("Starting the trusted setup process ...")
tau = random.randint(1, p)      # get a random number tau
d = 4                           # here we are working with a poly of degree 4
powers_of_tau_G1 = generate_powers_of_tau(tau, d, G1)   # Generate powers of tau [G1, [tau]G1, [tau^2]G1, [tau^3]G1, [tau^4]G1]
tauG2 = multiply(G2, int(tau))
print("tau = ", tau);
print("tau * G2 = ", tauG2);

Starting the trusted setup process ...
tau =  19465485706817622723382740072826716348161119121687213014771794665024780975491
tau * G2 =  ((11247395856097492827305355562510906332542136996059317800425636396561143801920, 20595588957877626158655871624825020802150564402002577065476861957365266790216), (792117968089077715308745982624365730219482899643508230916671037982546527533, 9065865693651718281807478072133264399215287995628978566984864934215491998657))


In [5]:
print("Committing poly f(X) ...")
fX = galois.Poly([5, 0, 0, - 2, 3], field = GF)
com_f = inner_product(powers_of_tau_G1, fX.coeffs[::-1])
print("Commitment com_f: ", com_f)

Committing poly f(X) ...
Commitment com_f:  (2185357591995161491282103273655962372364221723039242029771313900080707328922, 3506762793404286731729255492663835832293994265110975627805774775302582796222)


In [6]:
# Open the commitment
print("Opening the commitment ...")    
u = 2               # Verifier chooses a random v, e.g., v = 2, then sends it the prover
# Prover calculates v and the polynomial Q(X)
v = fX(u)           
qX = (fX - v) // galois.Poly([1, -u], field = GF)
print("Q(X): ", qX)

print("Generating proof ...")
com_q = inner_product(powers_of_tau_G1[:d], qX.coeffs[::-1])
print("Proof of commitment com_q: ", com_q)

Opening the commitment ...
Q(X):  5x^3 + 10x^2 + 20x + 38
Generating proof ...
Proof of commitment com_q:  (16023929903155862173621927744155775517826161507199864652232369747933717115609, 17481877964210517788679116335044914637384694359555431978687750926059849649081)


In [7]:
uG2 = multiply(G2, int(u))
tau_minus_uG2 = add(tauG2, neg(uG2))
print("tau_minus_uG2 = ", tau_minus_uG2)
vG1 = multiply(G1, int(v))
com_f_minus_vG1 = add(com_f, neg(vG1))
print("com_f_minus_vG1 = ", com_f_minus_vG1)

tau_minus_uG2 =  ((8974409604474593881424262899681016202238946847434369819191428729659100177173, 2100650450433163127448797638846340744415129157661808598372777820409759410835), (4610592300152902485926937644073547063427634653565133602877111544733900875444, 7278965958694509953290788541589832595257965218216299435472526660970378129316))
com_f_minus_vG1 =  (15587765004682323766697827624999550960437130827527031178183385184089657752373, 20938358244414769917382623485293067594596629172257767425997520479166692065293)


In [ ]:
pairing(tau_minus_uG2, com_q)

In [ ]:
if pairing(tau_minus_uG2, com_q) == pairing(G2, com_f_minus_vG1):
    print("Proof for commitment is correct!")
else:
    print("Failed to test the proof!")